In [1]:
import os
import sys
print(f"{sys.executable = }")
print(f"{os.getcwd() = }")
sys.path.append('/nas/home/sh0416/sentence-benchmark/src')


sys.executable = '/home/sh0416/anaconda3/envs/sentence-benchmark/bin/python'
os.getcwd() = '/nas/home/sh0416/sentence-benchmark/src/notebooks'


In [2]:
root = "/nas/home/sh0416/data/semeval16/task2/test_goldStandard"
model_args_path = "/home/sh0416/checkpoints/20211021_152037/model_args.json"
checkpoint_path = "/home/sh0416/checkpoints/20211021_152037/checkpoint-2000/pytorch_model.bin"
sources = ["images", "headlines", "answers-students"]

filepaths_sent1 = [f"STSint.testinput.{source}.sent1.txt" for source in sources]
filepaths_sent2 = [f"STSint.testinput.{source}.sent2.txt" for source in sources]
filepaths_sent1_tok = [f"STSint.testinput.{source}.sent1.chunk.txt" for source in sources]
filepaths_sent2_tok = [f"STSint.testinput.{source}.sent2.chunk.txt" for source in sources]
filepaths_align = [f"STSint.testinput.{source}.wa" for source in sources]

In [3]:
import os
from simcse.eval.ists import load_alignment
from simcse.eval.ists import save_alignment
data = load_alignment(os.path.join(root, filepaths_align[0]))
save_alignment(data, "test.wa")

In [12]:
from simcse.eval.ists import load_instances
instances = load_instances(
    os.path.join(root, filepaths_sent1[1]),
    os.path.join(root, filepaths_sent2[1]),
    os.path.join(root, filepaths_sent1_tok[1]),
    os.path.join(root, filepaths_sent2_tok[1])
)
instances[0]

{'id': 1,
 'sent1': "China 's Peace Ark departs for the Philippines Thursday",
 'sent2': 'China sends aid team to Philippines',
 'sent1_chunk': ['China',
  "'s Peace Ark",
  'departs',
  'for the Philippines',
  'Thursday'],
 'sent2_chunk': ['China', 'sends', 'aid team', 'to Philippines']}

In [13]:
import json
from transformers import AutoTokenizer
from simcse.config import ModelArguments

with open(model_args_path) as f:
    model_args = ModelArguments(**json.load(f))
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, use_fast=False)

In [14]:
model_args

ModelArguments(model_name_or_path='roberta-base', temp=0.05, pooler_type='cls', hidden_dropout_prob=0.1, mlp_only_train=True, coeff_mlm=0.1, loss_rwmd=True, layer_idx=12)

In [15]:
from simcse.eval.ists import preprocess_instances

prep_instances = preprocess_instances(tokenizer, instances)
prep_instances[0]

{'instance': {'id': 1,
  'sent1': "China 's Peace Ark departs for the Philippines Thursday",
  'sent2': 'China sends aid team to Philippines',
  'sent1_chunk': ['China',
   "'s Peace Ark",
   'departs',
   'for the Philippines',
   'Thursday'],
  'sent2_chunk': ['China', 'sends', 'aid team', 'to Philippines']},
 'sent1_token': ['China',
  "Ġ'",
  's',
  'ĠPeace',
  'ĠArk',
  'Ġdep',
  'arts',
  'Ġfor',
  'Ġthe',
  'ĠPhilippines',
  'ĠThursday'],
 'sent2_token': ['China', 'Ġsends', 'Ġaid', 'Ġteam', 'Ġto', 'ĠPhilippines'],
 'inputs1': {'input_ids': tensor([[    0,  8481,   128,    29,  7774, 19365,  8273,  7870,    13,     5,
           5639,   296,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 'inputs2': {'input_ids': tensor([[    0,  8481, 11210,  2887,   165,     7,  5639,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}}

In [16]:
import torch
from simcse.models import create_contrastive_learning

module = create_contrastive_learning(model_args)
module.load_state_dict(torch.load(checkpoint_path))
module.model

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SentenceSimilarityModel(
  (representation_model): LastHiddenSentenceRepresentationModel(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
     

In [17]:
from simcse.eval.ists import inference

infered_instances = inference(module.model, prep_instances)

In [18]:
infered_instances[0]["pairs"]

[{'sent1_word_ids': [1],
  'sent2_word_ids': [1],
  'type': 'EQUI',
  'score': 5,
  'comment': ''},
 {'sent1_word_ids': [6, 7, 8],
  'sent2_word_ids': [5, 6],
  'type': 'EQUI',
  'score': 5,
  'comment': ''}]

In [19]:
from simcse.eval.ists import save_infered_instances

save_infered_instances(infered_instances, f"{sources[1]}_{model_args.model_name_or_path}_rwmd-pool.wa")

In [ ]:
x = np.zeros((4, 2))
x[[0, 1], [0, 1]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_heatmap(sentence):
    source = ['0 NIL : '] + sentence["source"]
    translation = ['0 NIL : '] + sentence["translation"]
    heatmap = np.zeros((len(source), len(translation)))
    for a in sentence["alignment"]:
        if a["score"] != 'NIL':
            token_id_seq1, token_id_seq2 = a["pair_id"]
            score = float(a["score"])
            for token_id1 in token_id_seq1:
                for token_id2 in token_id_seq2:
                    heatmap[token_id1, token_id2] = score

    fig, ax = plt.subplots(facecolor='white')
    ax.set_ylabel("source")
    ax.set_yticks(np.arange(0.5, len(source)+0.5, 1))
    ax.set_yticklabels(source)
    ax.set_xlabel("translation")
    ax.set_xticks(np.arange(0.5, len(translation)+0.5, 1))
    ax.set_xticklabels(translation, rotation=90)
    ax.pcolormesh(heatmap, cmap=plt.get_cmap('Blues'))
    ax.invert_yaxis()
    for a in sentence["alignment"]:
        if a["type"] != 'NOALI':
            token_id_seq1, token_id_seq2 = a["pair_id"]
            for token_id1 in token_id_seq1:
                for token_id2 in token_id_seq2:
                    ax.text(token_id2, token_id1+0.5, a["type"], c='w')
    plt.tight_layout()
    return fig, ax

for sentence in data[:100]:
    plot_heatmap(sentence)
    plt.show()

In [ ]:
from simcse.models import create_contrastive_learning
from simcse.config import ModelArguments

model_args = ModelArguments(loss_rwmd=True)
module = create_contrastive_learning(model_args)